<h1 style="color: darkblue; font-family: 'Times New Roman', sans-serif;">
ECUACIÓN DE ESTADO DE REDLICH KWONG
</h1>

<h2 style="color: SlateBlue; font-family: 'Times New Roman', sans-serif;">
Calculo para una mezcla multicomponente
</h2>

In [1]:
# ECUACIÓN DE ESTADO DE REDLICH--KWONG PARA UNA MEZCLA MULTICOMPONENTE
import CoolProp.CoolProp as CP
import numpy as np
from scipy.optimize import newton
import requests
from PIL import Image
from io import BytesIO

In [2]:
# Constante de los gases
R = 8.314472  # J /mol K

print('-----ECUACIÓN DE ESTADO DE REDLICH-KWONG-----')

# Número de componentes
n = int(input('Ingrese el número de componentes de la mezcla: '))

# Listas para almacenar nombres de sustancias y propiedades críticas
nombres_sustancias = []
TC = []
PC = []

# Obtener propiedades críticas de cada sustancia usando CoolProp
for i in range(n):
    nombre_sustancia = input(f'Ingrese el nombre de la sustancia {i+1} en ingles: ')
    nombres_sustancias.append(nombre_sustancia)
    try:
        TC_i = CP.PropsSI('Tcrit', nombre_sustancia)
        PC_i = CP.PropsSI('Pcrit', nombre_sustancia) * 1e-5  # Convertir Pa a bar
        TC.append(TC_i)
        PC.append(PC_i)
        print(f'Sustancia seleccionada: {nombre_sustancia}')
        print(f'Temperatura crítica (TC): {TC_i} K')
        print(f'Presión crítica (PC): {PC_i} bar')
        print("")
    except Exception as e:
        print(f'Error: {e}')
        print('Sustancia no encontrada en la base de datos de CoolProp.')
        TC = float(input('Ingrese la temperatura critica de la sustancia en Kelvin : '))
        PC = float(input('Ingrese la Presion critica de la sustancia en bar : '))
        exit()
# Fracciones molares
x = []
for i in range(n):
    xi = float(input(f'Ingrese la fracción molar de la sustancia {nombres_sustancias[i]}: '))
    x.append(xi)

# Verificar que la suma de las fracciones molares sea 1
if not np.isclose(sum(x), 1.0):
    print('Las fracciones molares no suman 1.')
    exit()

# Parámetros de cada sustancia de la ecuación de Redlich-Kwong
a = []
b = []
for i in range(n):
    a_i = (9*((2**(1/3))-1))**(-1) * ((R**2 * TC[i]**2.5)/(PC[i]*1e5))
    a_i = round(a_i, 5)
    b_i = (((2**(1/3))-1)/3) * ((R * TC[i]) / (PC[i]*1e5))
    a.append(a_i)
    b.append(b_i)

# Parámetros de la mezcla utilizando la regla de mezclado
print("")
a_mix = sum(x[i] * x[j] * (a[i] * a[j])**0.5 for i in range(n) for j in range(n))
b_mix = sum(x[i] * b[i] for i in range(n))
print(' amix = ', a_mix, ' [(J m^3 K^0.5) / mol^2] ')
print(' bmix = ', format(b_mix, '0.5e'), ' [m^3 / mol] ')
print("")

# Elección de la variable a calcular
tipo = input('''Elija la variable que desea calcular :
 Teclear P si va a calcular la presión
 Teclear T si va a calcular la temperatura
 Teclear v si va a calcular el volumen específico \n''')

# Cálculo de presión
if tipo.lower() == 'p':
    T = float(input('Temperatura (en Kelvin): '))
    v = float(input('Volumen específico (en m^3/mol): '))
    P = ((R * T)/(v - b_mix)) - (a_mix / (T**0.5 * v * (v + b_mix)))
    P = round(P, 2)
    print(' La presión es igual a ', P, ' Pa   =  ', round(P/100000, 3), ' bar ')

    resultados = [P]

# Cálculo de temperatura
elif tipo.lower() == 't':
    P = float(input('Presión (en bar): '))
    P = P * 100000
    v = float(input('Volumen específico (en m^3/mol): '))
    m1 = -(P * (v - b_mix)) / R
    m2 = -a_mix / (R * v)

    def f(T):
        return T**(3/2) + (m1 * T**0.5) + m2
    T0 = newton(f, max(TC), fprime=None, args=(), tol=1.5e-08, maxiter=50, fprime2=None)
    T0 = round(T0.real, 2)
    print(' La temperatura es igual a ', T0, ' K ')
    resultados = [T0]

# Cálculo de volumen
elif tipo.lower() == 'v':
    P = float(input('Presión (en bar): '))
    P = P * 100000
    T = float(input('Temperatura (en Kelvin): '))

    c1 = 1
    c2 = -(R*T)/P
    c3 = (a_mix / (P * T**0.5)) - (b_mix**2) - ((b_mix * R * T) / P)
    c4 = -(a_mix*b_mix)/(P * T**0.5)
    coeff = [c1, c2, c3, c4]
    v = np.roots(coeff)
    print('v1 = ', np.round(v[0], 6), '[m^3/mol] ')
    print('v2 = ', np.round(v[1], 8), '[m^3/mol] ')
    print('v3 = ', np.round(v[2], 8), '[m^3/mol] ')

    resultados = [v]

else:
    print(' No eligió una variable válida')


-----ECUACIÓN DE ESTADO DE REDLICH-KWONG-----


Ingrese el número de componentes de la mezcla:  3
Ingrese el nombre de la sustancia 1 en ingles:  Hydrogen


Sustancia seleccionada: Hydrogen
Temperatura crítica (TC): 33.145 K
Presión crítica (PC): 12.964 bar



Ingrese el nombre de la sustancia 2 en ingles:  Water


Sustancia seleccionada: Water
Temperatura crítica (TC): 647.096 K
Presión crítica (PC): 220.64000000000001 bar



...

Sustancia seleccionada: Ethane
Temperatura crítica (TC): 305.322 K
Presión crítica (PC): 48.722 bar



Ingrese la fracción molar de la sustancia Hydrogen:  0.5
Ingrese la fracción molar de la sustancia Water:  0.4
Ingrese la fracción molar de la sustancia Ethane:  0.1



 amix =  4.060344934777683  [(J m^3 K^0.5) / mol^2] 
 bmix =  2.21739e-05  [m^3 / mol] 



Elija la variable que desea calcular :
 Teclear P si va a calcular la presión
 Teclear T si va a calcular la temperatura
 Teclear v si va a calcular el volumen específico 
 p
Temperatura (en Kelvin):  300
Volumen específico (en m^3/mol):  0.00125


 La presión es igual a  1884094.12  Pa   =   18.841  bar 


In [9]:
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
import openai


#Asignar la clave API y crear cliente para chatty
load_dotenv("xxx.env")

# Recupera la API key
api_key = os.getenv("OPENAI_API_KEY")
if api_key :
    print("Acceso permitido")
else:
    print("No se encontró la clave")

client = openai.OpenAI(api_key=api_key)

Acceso permitido


In [6]:

response = client.chat.completions.create(
    model ="gpt-4o",
    messages =[
        {"role": "system", "content": "Eres un asitente util, sabes de termodinámica y ecuaciónes de estado, vas directamente a lo que el usuario te pide, eres claro y conciso, evita el uso de codigo latex"},
        {"role": "user", "content": f'''Obtuve estos resultados:
        ECUACIÓN DE ESTADO DE REDLICH-KWONG
        Las sustancias son las siguientes: {[nombres_sustancias]}
        Fracciones molares: {[x]}
        y la propiedad de la mezcla es la siguiente:
        amix = {a_mix} [(J m^3 K^0.5) / mol^2] 
        bmix = {b_mix} [m^3 / mol]
        Donde la mezcla esta en las siguientes condiciones:
        Temperatura: {T} kelvin
        Volumen especifico: {v} m3/mol
        Se obtuvieron los siguientes resultados: {resultados}
        ¿Que puedes concluir de la presión obtenida, en base a las condicines es una respuesta valida?
        '''}
    ],
    #Establecer limite de tokens
    #max_tokens=200 
)

respuesta = (response.choices[0].message.content)

display(Markdown(respuesta))

Para determinar si la presión calculada con la ecuación de estado de Redlich-Kwong es válida, debemos considerar las condiciones dadas: 

1. **Composición de la mezcla:** La mezcla consiste en hidrógeno, agua y etano con fracciones molares de 0.5, 0.4, y 0.1 respectivamente. Esta composición puede influir en el comportamiento del gas debido a las diferencias en las propiedades de estos componentes, como peso molecular y polaridad.

2. **Temperatura y volumen específico:** La temperatura de 300 K y el volumen específico de 0.00125 m^3/mol indican una densidad relativamente alta. Esto es importante porque las ecuaciones cúbicas de estado como Redlich-Kwong tienden a ser menos precisas a alta densidad, especialmente cerca de los puntos críticos.

3. **Presión calculada:** La presión obtenida es de aproximadamente 1,884,094.12 Pa, que equivale aproximadamente a 18.84 bar. Esta es una presión considerablemente alta, sugiriendo que el gas está en una condición de alta presión y posiblemente en la fase líquida o cerca de una región de cambio de fase. Esto es consistente con la alta densidad de la mezcla.

En resumen, la presión calculada parece razonable dadas las condiciones de temperatura y volumen específico. Sin embargo, la validez absoluta de este resultado dependerá de la adecuación de la ecuación de estado de Redlich-Kwong para describir el comportamiento de la mezcla en estas condiciones. También, en alta presión, la interacción molecular y posibles desviaciones de idealidad son significativas, lo cual la ecuación de Redlich-Kwong trata de capturar mediante sus parámetros para amix y bmix. Recomendablemente, verifica si hay datos experimentales disponibles o calcula la presión utilizando otras ecuaciones de estado o modelos para asegurarte de la precisión del resultado.